![ETL](src/PORT2.jpg)

# **ETL del dataset:** *steam_games*

#### **Importación de librerías** ####
---

*Para este cuaderno usaremos las siguientes librerías: **pandas, gzip, pyarrow y numpy**.*

In [1]:
import pandas as pd
import gzip
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

#### **Extracción de los datos** ####
---

*Procedemos a abrir nuestro dataset y almacenarlo en un dataframe para visualizarlo y manejarlo en pandas.*

1. *Usamos **with** para asegurar un correcto funcionamiento de la apertura y cierre del archivo.*
2. *Usamos **gzip** para abrir archivos comprimidos en modo de lectura bin.*
3. *Usamos el **.read_json**, metodo de pandas para leer archivos tipo json.*

In [2]:
with gzip.open("datasets\steam_games.json.gz", "rb") as jsonFile:
    dfSteam = pd.read_json(jsonFile, lines=True)

### **Análisis Exploratorio Inicial** ###
---

*Vamos a realizar una **exploración de los datos** (EDA) antes de hacer transformación de los mismos, esto con el fin de **conocer las columnas** y determinar que campos son valiosos para nuestro análisis y así tener claro que pasos realizaremos en el ETL.*

**Nota:**: *Posterior al ETL realizaremos también un [EDA](04_EDA.ipynb)
 enfocado en los datos y una descripción estadística.*

#### **Revisión de la extracción de los datos** ####

*Revisamos que los datos se cargaron adecuadamente en nuestro dataframe:*

1. *Usamos **head** para previsualizar los primeros registros del Dataframe.*
2. *Podemos observar que se cargaron adecuadamente los datos, pero vemos datos **None**.*

In [3]:
dfSteam.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


*Revisemos si este patrón se repite en todos los datos:*

1. *Usamos **tail** para previsualizar los últimos registros del Dataframe.*
2. *Observamos que los últimos registros del Dataframe si tienen **datos "válidos"**.*
3. *A vista general podemos observar que en nuestro Dataframe existen **datos nulos**, pero esto lo revisaremos más adelante.*

In [4]:
dfSteam.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


*Revisamos la forma del dataframe:*

1. *Usamos **shape** obtener el número de filas y columnas del dataframe.*
2. *Nuestro dataframe tiene **120445 filas y 13 columnas**.*

In [5]:
dfSteam.shape

(120445, 13)

#### **Exploración de las columnas** ####

*Vamos a revisar qué columnas existen en el dataframe y de qué tipo son:*

1. *Usamos el método **dtypes** para listar las columnas y sus tipos.*
2. *Observamos que 11 columnas contienen cadenas de caracteres identificados como **object** y 2 columnas son numéricas identificados como **float**.*

In [6]:
dfSteam.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price            object
early_access    float64
id              float64
developer        object
dtype: object

*Revisemos qué tipos de datos almacena cada columna a detalle:*

1. *Hacemos una **iteración** que nos permite recorrer los valores de cada columna para encontrar sus tipos.*
2. *Observamos que la columna **Price tiene dos tipos de datos**, por lo que debemos tenerlo en cuenta al momento de hacer la transformación.*
3. *En las demás columnas vemos la presencia de datos **Nulos**.*
4. *La columna genres, tags y specs son **listas**.*

In [7]:
tipoDatos = {"Columna":[], "Tipos":[]}
for c in dfSteam.columns:
    tipoDatos["Columna"].append(c)
    tipoDatos["Tipos"].append(dfSteam[c].apply(type).unique())
    
dfTypes = pd.DataFrame(tipoDatos)
dfTypes

,Columna,Tipos
0,publisher,"[<class 'NoneType'>, <class 'str'>]"
1,genres,"[<class 'NoneType'>, <class 'list'>]"
2,app_name,"[<class 'NoneType'>, <class 'str'>]"
3,title,"[<class 'NoneType'>, <class 'str'>]"
4,url,"[<class 'NoneType'>, <class 'str'>]"
5,release_date,"[<class 'NoneType'>, <class 'str'>]"
6,tags,"[<class 'NoneType'>, <class 'list'>]"
7,reviews_url,"[<class 'NoneType'>, <class 'str'>]"
8,specs,"[<class 'NoneType'>, <class 'list'>]"
9,price,"[<class 'NoneType'>, <class 'float'>, <class '..."


*Revisemos las columnas consideradas como numéricas:*

1. *Usamos **describe()** para obtener un resumen de estas columnas.*
2. *Por ahora no dicen mucho sobre nuestros datos ya que una es una variable **booleana** y la otra un **identificador**.*

In [8]:
dfSteam.describe()

,early_access,id
count,32135.000000,3.213300e+04
mean,0.060588,4.517574e+05
std,0.238577,1.827140e+05
min,0.000000,1.000000e+01
25%,0.000000,3.292800e+05
50%,0.000000,4.520600e+05
75%,0.000000,5.934000e+05
max,1.000000,2.028850e+06


*Revisemos los valores alojados en cada columna para saber cuantos son validos, es decir no nulos:*

1. *Usamos **info()** para obtener un conteo de los valores válidos por columnas*
2. *Por ahora podemos observar que tenemos una alta presencia de valores nulos. Un aproximado de **9000** registros.*

In [9]:
dfSteam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


*Revisemos las columnas en porcentajes de valores nulos:*

1. *Usamos **isnull()** y **sum()** para obtener una suma de los valores nulos por columna.*
2. *Observamos que entre el **70% y 80%** de los regitros de cada columna son vacíos.*

In [10]:
nulosPorCol = dfSteam.isnull().sum()
porcentajePorCol = (nulosPorCol/dfSteam.shape[0])*100
porcentajePorCol

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

*Descripción de las variables del dataframe:*

1. *Creamos una tabla para **describir y comparar las columnas**.*

| Variable     | Definición                                               |Tipo de Dato| Tipo de Variable               |
|:-------------|:---------------------------------------------------------|:----------:|:-------------------------------|
| Publisher    | Nombre de la empresa desarrolladora de videojuegos       | String     | Categórica(nominal)            |
| Genres       | Son los generos en los que está etiquetado un videojuego | List       | Categórica(nominal)            |
| App_name	   | Nombre de la aplicación                                  | String     | Categórica(nominal)            |
| Title        | Título de la aplicación                                  | String     | Categórica(nominal)            |
| Url          | Enlace al videojuego                                     | String     | Referencia a una dirección web |
| Release Date | Fecha en que se lanzó                                    | String     | Catégorica(ordinal)            |
| Tags         | Etiquetas del videojuego                                 | List       | Catégorica(nominal)            |
| Reviews_url  | Enlace a las reviews del videojuego                      | String     | Referencia a una dirección web |
| Specs        | Características del videojuego                           | List       | Categórica(nominal)            |
| Price        | Precio del videojuego                                    | Float      | Numérica Continua              |
| Early_access | Acceso Prelanzamiento / Beta version                     | Boolean    | Dicótomica / Numérica discreta |
|       Id     | Identificador único del videojuego                       | Integer    | Numérica Discreta              |
|  Developer   | Nombre desarrollador del videojuego                      | String     | Catégorica(nominal)            |

2. *Revisamos las especificaciones de las funciones que requiere nuestro **MVP** para saber que columnas serán importantes para las funcionalidades de la API.*
3. *Observamos que existen columnas como **Url y Reviews url** que no serán apropiadas para los objetivos de este análisis.*
4. *Observamos que **App Name y Title** pueden ser columnas similares, así que lo revisaremos más adelante para no tener información duplicada.*
5. *Observamos que **Publisher y Developer** pueden ser columnas similares, así que lo revisaremos más adelante.*

**Nota:** *En algunos visualizadores de Markdown no es posible apreciar la tabla, sugiero revisarla en VSC.*

### **Transformación de los datos** ###
---

*A partir de este momento realizaremos una limpieza de los datos teniendo en cuenta nuestro análisis previo.*

* **Nota:** *Posterior al ETL realizaremos también un [EDA](04_EDA.ipynb) enfocado en los datos y una descripción estadística.*

#### Eliminación de datos nulos ####

*Procedemos a eliminar las filas con registros nulos:*

1. *Usamos **dropna()** para eliminar filas con registros nulos*.
2. *El número de filas cambió a **22530**, recordemos que antes teníamos más de 120520 registros*.

In [11]:
dfSteam = dfSteam.dropna()
dfSteam.shape

(22530, 13)

*Revisemos qué cambios hubo:*

1. *Efectivamente **eliminamos las filas con valores nulos**.*
2. *Tenemos un **0% de valores nulos** por columna.*

In [12]:
dfSteam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22530 non-null  object 
 1   genres        22530 non-null  object 
 2   app_name      22530 non-null  object 
 3   title         22530 non-null  object 
 4   url           22530 non-null  object 
 5   release_date  22530 non-null  object 
 6   tags          22530 non-null  object 
 7   reviews_url   22530 non-null  object 
 8   specs         22530 non-null  object 
 9   price         22530 non-null  object 
 10  early_access  22530 non-null  float64
 11  id            22530 non-null  float64
 12  developer     22530 non-null  object 
dtypes: float64(2), object(11)
memory usage: 2.4+ MB


#### Eliminación de duplicados de datos ####

*Procedemos a revisar si nos quedan filas cuyos valores sean duplicados:*

1. *Usamos **duplicated()** para encontrar filas con registros nulos usando la columna "id"*.
2. *Observamos que nos muestra un registro cuyo id es **612880**.*

In [13]:
duplicados = dfSteam[dfSteam.duplicated(subset="id")]
duplicados

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


*Buscamos el id que nos identificó pandas:*

1. *Usamos **loc()** para encontrar filas con "id" 612880*.
2. *Observamos que nos muestra **dos registros identicos**.*

In [14]:
dfSteam.loc[dfSteam["id"]== 612880]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


*Eliminamos el registro:*

1. *Usamos **drop()** para eliminar la fila con indice 102883*.
2. *Volvemos a **verificar** si quedan registros duplicados*.

In [15]:
dfSteam = dfSteam.drop(102883)

In [16]:
duplicados = dfSteam[dfSteam.duplicated(subset="id")]
duplicados

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


#### Tranformación por columnas ####

*A continuación vamos a hacer limpieza por columnas, pero antes haremos algunos cambios:*

1. **Reorganizamos las columnas** *, con el fin de mejorar la lectura y comparación de los datos y columnas que consideramos similares en el análisis anterior*.
2. **Cambiamos los nombres** *de las columnas.*

In [17]:
dfSteam = dfSteam[["id", "app_name", "title","developer", "publisher", "genres", "tags", "specs", "early_access",
                   "release_date","price", "url", "reviews_url"]]

In [18]:
dfSteam.columns = ['IdApp', 'Name', "Title", "Developer", "Publisher", "Genres", "Tags", "Specs", "EarlyAccess",
                   "ReleaseDate", "Price", "Url", "UrlReviews"]

**Columna IdApp**

1. *Cambiamos el tipo de dato a **int**.*
2. *Reorganizamos los datos por **id**.*

In [19]:
dfSteam["IdApp"] = dfSteam["IdApp"].astype(np.int64)

In [20]:
dfSteam.sort_values(by="IdApp", inplace=True)
dfSteam.reset_index(inplace=True, drop=True)
dfSteam.head(3)

,IdApp,Name,Title,Developer,Publisher,Genres,Tags,Specs,EarlyAccess,ReleaseDate,Price,Url,UrlReviews
0,10,Counter-Strike,Counter-Strike,Valve,Valve,[Action],"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",0.0,2000-11-01,9.99,http://store.steampowered.com/app/10/CounterSt...,http://steamcommunity.com/app/10/reviews/?brow...
1,20,Team Fortress Classic,Team Fortress Classic,Valve,Valve,[Action],"[Action, FPS, Multiplayer, Classic, Shooter, C...","[Multi-player, Valve Anti-Cheat enabled]",0.0,1999-04-01,4.99,http://store.steampowered.com/app/20/Team_Fort...,http://steamcommunity.com/app/20/reviews/?brow...
2,30,Day of Defeat,Day of Defeat,Valve,Valve,[Action],"[FPS, World War II, Multiplayer, Action, Shoot...","[Multi-player, Valve Anti-Cheat enabled]",0.0,2003-05-01,4.99,http://store.steampowered.com/app/30/Day_of_De...,http://steamcommunity.com/app/30/reviews/?brow...


**Columnas Name y Title**

1. *Vamos a **verificar** si los valores entre las dos columnas son iguales*.
2. *Creamos una **máscara** para filtrar valores que sean diferentes*
3. *De las **22529** filas solo **351** son diferentes en este campo*
4. *Pero revisemos bien, **los campos de las dos columnas en si son iguales**, solo hay un error en la interpretación del carácter &*
5. *Por lo tanto procedemos a eliminar la columna **Title** para no tener información duplicada*

In [21]:
titNameDiferentes = dfSteam['Title'] != dfSteam['Name']
filasDiferentes = dfSteam[titNameDiferentes]

In [22]:
filasDiferentes.head(5)

,IdApp,Name,Title,Developer,Publisher,Genres,Tags,Specs,EarlyAccess,ReleaseDate,Price,Url,UrlReviews
46,2100,Dark Messiah of Might & Magic,Dark Messiah of Might &amp; Magic,Arkane Studios,Ubisoft,"[Action, RPG]","[RPG, Action, First-Person, Fantasy, Adventure...","[Single-player, Multi-player, Valve Anti-Cheat...",0.0,2006-10-25,9.99,http://store.steampowered.com/app/2100/Dark_Me...,http://steamcommunity.com/app/2100/reviews/?br...
180,6250,Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm,Muzzy Lane,Strategy First,"[Action, Strategy]","[Strategy, Action, Turn-Based Strategy, Grand ...","[Single-player, Multi-player]",0.0,2007-03-13,4.99,http://store.steampowered.com/app/6250/Making_...,http://steamcommunity.com/app/6250/reviews/?br...
231,8200,Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock,Telltale Games,Telltale Games,"[Action, Adventure]","[Point & Click, Comedy, Adventure, Detective, ...",[Single-player],0.0,2006-10-17,19.99,http://store.steampowered.com/app/8200/Sam__Ma...,http://steamcommunity.com/app/8200/reviews/?br...
232,8210,Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy,Telltale Games,Telltale Games,"[Action, Adventure]","[Adventure, Action]",[Single-player],0.0,2006-12-20,19.99,http://store.steampowered.com/app/8210/Sam__Ma...,http://steamcommunity.com/app/8210/reviews/?br...
233,8220,"Sam & Max 103: The Mole, the Mob and the Meatball","Sam &amp; Max 103: The Mole, the Mob and the M...",Telltale Games,Telltale Games,"[Action, Adventure]","[Adventure, Action, Point & Click]",[Single-player],0.0,2007-01-25,19.99,http://store.steampowered.com/app/8220/Sam__Ma...,http://steamcommunity.com/app/8220/reviews/?br...


In [23]:
dfSteam = dfSteam.drop(columns="Title")

In [24]:
dfSteam["Name"] = dfSteam["Name"].astype(str)

**Columnas Developer y Publisher**

1. *Vamos a **verificar** si los valores entre las dos columnas son iguales*.
2. *Creamos una **máscara** para filtrar valores que sean diferentes.*
3. *De las **22529** filas **11001** son diferentes en este campo.*
4. *Por lo tanto son columnas con valores diferentes que en muy pocos casos se repiten y se deben mantener ambas columnas*.

In [25]:
devPubDiferentes = dfSteam['Developer'] != dfSteam['Publisher']
filasDevPubDiferentes = dfSteam[devPubDiferentes]

In [26]:
filasDevPubDiferentes

,IdApp,Name,Developer,Publisher,Genres,Tags,Specs,EarlyAccess,ReleaseDate,Price,Url,UrlReviews
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,[Action],"[FPS, Action, Sci-fi, Singleplayer, Classic, S...","[Single-player, Multi-player, Valve Anti-Cheat...",0.0,1999-11-01,4.99,http://store.steampowered.com/app/50/HalfLife_...,http://steamcommunity.com/app/50/reviews/?brow...
8,130,Half-Life: Blue Shift,Gearbox Software,Valve,[Action],"[FPS, Action, Sci-fi, Singleplayer, Shooter, A...",[Single-player],0.0,2001-06-01,4.99,http://store.steampowered.com/app/130/HalfLife...,http://steamcommunity.com/app/130/reviews/?bro...
26,1230,Mare Nostrum,Sandstorm Productions,Tripwire Interactive,[Action],"[Action, World War II, FPS, Mod, Multiplayer, ...","[Multi-player, Mods, Steam Achievements]",0.0,2008-10-17,Free,http://store.steampowered.com/app/1230/Mare_No...,http://steamcommunity.com/app/1230/reviews/?br...
36,1600,Dangerous Waters,Sonalysts,Strategy First,[Strategy],"[Strategy, Simulation, Naval]","[Single-player, Multi-player]",0.0,2006-02-07,14.99,http://store.steampowered.com/app/1600/Dangero...,http://steamcommunity.com/app/1600/reviews/?br...
37,1610,Space Empires IV Deluxe,Malfador Machinations,Strategy First,[Strategy],"[Strategy, 4X, Sci-fi, Turn-Based, Turn-Based ...","[Single-player, Multi-player]",0.0,2006-02-07,9.99,http://store.steampowered.com/app/1610/Space_E...,http://steamcommunity.com/app/1610/reviews/?br...
...,...,...,...,...,...,...,...,...,...,...,...,...
22524,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,"Ubisoft Paris,Red Storm Entertainment",Ubisoft,[Action],[Action],"[Single-player, Multi-player, Co-op, Downloada...",0.0,2012-10-25,24.99,http://store.steampowered.com/app/2028055/Tom_...,http://steamcommunity.com/app/2028055/reviews/...
22525,2028056,Worms Revolution Season Pass,Team17 Digital Ltd.,Team17 Digital Ltd,[Strategy],[Strategy],"[Single-player, Multi-player, Co-op, Shared/Sp...",0.0,2012-10-10,14.99,http://store.steampowered.com/app/2028056/Worm...,http://steamcommunity.com/app/2028056/reviews/...
22526,2028062,Call of Duty®: Black Ops II Season Pass,Treyarch,Activision,[Action],"[Action, FPS, Zombies, Multiplayer]","[Single-player, Multi-player, Co-op, Downloada...",0.0,Feb 2013,49.99,http://store.steampowered.com/app/2028062/Call...,http://steamcommunity.com/app/2028062/reviews/...
22527,2028103,Assassin’s Creed® III Season Pass,Ubisoft Montreal,Ubisoft,"[Action, Adventure]","[Action, Adventure]","[Single-player, Multi-player, Downloadable Con...",0.0,2012-11-20,29.99,http://store.steampowered.com/app/2028103/Assa...,http://steamcommunity.com/app/2028103/reviews/...


**Columna Genres**

1. *Tal como lo vimos en el EDA inicial la columna Genres está compuesta por listas*.
2. *Se toma la decisión de crear un dataframe que contenga la información del IdApp y sus generos mediante valores dummies.*

In [27]:
dfGenres = dfSteam[["IdApp", "Genres"]]

*Se crea una lista con los generos únicos del dataframe:*

In [28]:
listaGeneros = []
for e in dfGenres["Genres"]:
    for k in e:
        listaGeneros.append(k)
listaGeneros = sorted(list(set(listaGeneros)))

listaGeneros

['Action',
 'Adventure',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing']

*Creamos las columnas con cada uno de los generos:*

1. **Iteramos** *en la lista generos que creamos*
2. *Aplicamos una **función lambda** que determina si el genero está presente en la columna genres, sí está presente ingresa un valor de 1, si no lo está ingresa un valor de 0*.
3. *Revisamos que se ejecutó correctamente y **eliminamos la columna genres**.*
4. *Renombramos unas columnas.*

In [30]:
for genero in listaGeneros:
    dfGenres[genero] = dfGenres['Genres'].apply(lambda valor: 1 if genero in valor else 0)

In [31]:
dfGenres.iloc[60:65]

,IdApp,Genres,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
60,2390,[Action],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,2400,"[Action, Indie, RPG]",1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
62,2420,"[Action, Indie, RPG]",1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
63,2450,[Action],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,2500,[Action],1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
dfGenres.drop(columns=["Genres"],inplace=True)
dfGenres.rename(columns={"Animation &amp; Modeling":"Animation & Modeling",
                         "Design &amp; Illustration":"Design & Illustration"},inplace=True)

*Ahora eliminamos la columna genres de nuestro dataframe principal:*

In [33]:
dfSteam.drop(columns=["Genres"],inplace=True)
dfSteam.head(1)

,IdApp,Name,Developer,Publisher,Tags,Specs,EarlyAccess,ReleaseDate,Price,Url,UrlReviews
0,10,Counter-Strike,Valve,Valve,"[Action, FPS, Multiplayer, Shooter, Classic, T...","[Multi-player, Valve Anti-Cheat enabled]",0.0,2000-11-01,9.99,http://store.steampowered.com/app/10/CounterSt...,http://steamcommunity.com/app/10/reviews/?brow...


**Columnas Tags, Specs, EarlyAccess, Url, UrlReviews**

1. *Para efectos de este ejercicio estas columnas no serán necesarias para el **MPV**.*
2. *Por lo tanto se decide **almacenar estas columnas en otro Dataframe** y si es pertinente su consulta más adelante tener acceso a la información.*

In [34]:
dfMetadataSteam = dfSteam[["IdApp", "Tags", "Specs", "EarlyAccess", "Url", "UrlReviews"]].copy()

In [35]:
dfSteam.drop(columns=["Tags", "Specs", "EarlyAccess", "Url", "UrlReviews"],inplace=True)

**Columna Release Date**

1. *Observamos que esta columna tiene las fechas de lanzamiento de cada aplicación en **formato AÑO-MES-DIA**, aunque existen algunas excepciones*.
2. *Para efectos del análisis se decide **dividir esta columna en dos**, una columna con el año de lanzamiento y otra columna con el mes de lanzamiento*.
3. *Aplicamos un **iterador** que enumere la columna ReleaseDate, determinando lo siguiente:*

    - *Sí el largo de la fecha es de **10 caracteres (YYYY-MM-DD)**, entonces los primero cuatro caracteres serán almacenandos en la columna año y los caracteres en la posición 5 y 6 se almacenarán como mes.*
    - *Sí el largo de la fecha es de **8 caracteres (Feb 2023)**, entonces los4 utlimos caracteres serán almacenados en la columna año y los 3 primeros caracteres en la columna mes.*
    - *De no cumplirse las dos condiciones anteriores, el año y el mes serán **Sin Datos**.*

In [36]:
for i, e in enumerate(dfSteam["ReleaseDate"]):
    if len(e) == 10:
        dfSteam.loc[i, "ReleaseYear"] = e[0:4]
        dfSteam.loc[i, "ReleaseMonth"] = e[5:7]
    elif len(e) == 8:
        dfSteam.loc[i, "ReleaseYear"] = e[4:]
        dfSteam.loc[i, "ReleaseMonth"] = e[:3]
    else:
        dfSteam.loc[i, "ReleaseYear"] = "Sin Datos"
        dfSteam.loc[i, "ReleaseMonth"] = "Sin Datos"

*Aplicamos un iterador que enumere la columna ReleaseMonth, determinando lo siguiente:*

    - En cada caso se reemplaza el **mes** por el numero de mes.

In [37]:
for i, e in enumerate(dfSteam["ReleaseMonth"]):
    match e:
        case "Jan": dfSteam.loc[i, "ReleaseMonth"] = "01"
        case "Feb": dfSteam.loc[i, "ReleaseMonth"] = "02"
        case "Mar": dfSteam.loc[i, "ReleaseMonth"] = "03"
        case "Apr": dfSteam.loc[i, "ReleaseMonth"] = "04"
        case "May": dfSteam.loc[i, "ReleaseMonth"] = "05"
        case "Jun": dfSteam.loc[i, "ReleaseMonth"] = "06"
        case "Jul": dfSteam.loc[i, "ReleaseMonth"] = "07"
        case "Aug": dfSteam.loc[i, "ReleaseMonth"] = "08"
        case "Sep": dfSteam.loc[i, "ReleaseMonth"] = "09"
        case "Oct": dfSteam.loc[i, "ReleaseMonth"] = "10"
        case "Nov": dfSteam.loc[i, "ReleaseMonth"] = "11"
        case "Dec": dfSteam.loc[i, "ReleaseMonth"] = "12"

*Revisamos que las transformaciones de los datos se aplicaron:*

1. *Las transformaciones fueron **realizadas**.*
2. *Procedemos a **eliminar** la columna DateRelease.*

In [38]:
listaMeses = []
for e in dfSteam["ReleaseMonth"]:
    listaMeses.append(e)
listaMeses = sorted(list(set(listaMeses)))
print(listaMeses)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 'Sin Datos']


In [39]:
listaAños = []
for e in dfSteam["ReleaseYear"]:
    listaAños.append(e)
listaAños = sorted(list(set(listaAños)))
print(listaAños)

['1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2021', 'Sin Datos']


In [40]:
dfSteam.drop(columns="ReleaseDate", inplace=True)

**Columna Price**

1. *Observamos que esta columna tiene valores de precios, pero algunos son **Free** en diferentes formatos de texto.*.
2. *Vamos a **estandarizar todas las filas** que sus campos son: Free, Play Now, Demo, Install now y los reemplazaromos por Price = 0.00*.
3. *Convertimos el tipo de dato de la columna en **String** para poder manipular los datos.*

In [41]:
dfSteam["Price"] = dfSteam["Price"].astype(str)

*Iteramos la columna Price:*

1. *Sí el primer caracter de cada registro no es un digito, entonces significa que su precio es **Free**.*
2. *Reemplazamos el valor por **0.00**.*

In [42]:
for i, e in enumerate(dfSteam["Price"]):
    if e[0].isdigit() == False:
        dfSteam.loc[i,"Price"] = "0.00"

*Convertimos la columna en tipo Float:*
1. *Redondeamos los valores a **2 decimales**.*

In [43]:
dfSteam["Price"] = dfSteam["Price"].astype(float)

In [44]:
dfSteam["Price"] = round(dfSteam["Price"],2)

*Creamos una lista de los valores unicos para revisar si quedó algún valor pendiente por cambiar:*

1. *Todas las filas quedaron con valores adecuados*.

In [ ]:
listPrices = []
for p in dfSteam["Price"]:
    listPrices.append(p)
    
listPrices= list(set(listPrices))
listPrices

### **Análisis Exploratorio Final** ###
---

*Aquí hacemos una revisión de los datos después de su transformación, con el objetivo de revisar si quedaron datos pendientes por transformar*.

**Nota:** *Posterior a todas las transformaciones se realiza un EDA final, presente en el archivo EDA.ipynb*

*Obtenemos una descripción de las columnas, sus tipos de datos y sus valores no nulos:*

1. *El total de registros es de **22529**, y no se encuentran valores nulos*

In [46]:
dfSteam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22529 entries, 0 to 22528
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IdApp         22529 non-null  int64  
 1   Name          22529 non-null  object 
 2   Developer     22529 non-null  object 
 3   Publisher     22529 non-null  object 
 4   Price         22529 non-null  float64
 5   ReleaseYear   22529 non-null  object 
 6   ReleaseMonth  22529 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ MB


*Obtenemos una descripción estadística de los datos numéricos:*

1. *El promedio de precio de una app es de **8.96**.*
2. *El valor minimo de una app es de **0.00**.*
3. *El valor maximo de una app es de **995.00**.*

In [47]:
dfSteam.describe()

,IdApp,Price
count,2.252900e+04,22529.000000
mean,4.307083e+05,8.965943
std,1.918428e+05,15.414846
min,1.000000e+01,0.000000
25%,3.022900e+05,2.990000
50%,4.269280e+05,4.990000
75%,5.807000e+05,9.990000
max,2.028850e+06,995.000000


In [48]:
dfSteam.columns

Index(['IdApp', 'Name', 'Developer', 'Publisher', 'Price', 'ReleaseYear',
       'ReleaseMonth'],
      dtype='object')

### **Exportación de datos a Parquet (Load)** ###
---

*Una vez hemos revisado nuestro dataframe, lo podemos exportar a un archivo **CSV** para guardar todos los cambios realizados*.

*Sin embargo los archivos se guardaron en formato **parquet** para facilitar su manejo, ya que los archivos tipo parquet ocupan menos espacio.*

**Nota:** *Los archivos se encuentran en la carpeta dataout.*

In [478]:
#dfSteam.to_csv("datasets/out_steam_games.csv", index=False, encoding="utf-8")
#dfGenres.to_csv("datasets/out_steam_genres.csv", index=False, encoding="utf-8")
#dfMetadataSteam.to_csv("datasets/out_steam_metadata.csv", index=False, encoding="utf-8")

In [52]:
gamesTab = pa.Table.from_pandas(dfSteam)
dir = "dataout/out_games.parquet"
pq.write_table(gamesTab,dir)

In [53]:
genresTab = pa.Table.from_pandas(dfGenres)
dir2 = "datasout/out_genres_games.parquet"
pq.write_table(genresTab,dir2)

In [54]:
metaTab = pa.Table.from_pandas(dfMetadataSteam)
dir3 = "dataout/out_metadata_games.parquet"
pq.write_table(metaTab,dir3)